## Python timeit 库详解
　　参考：[python之timeit模块](https://www.cnblogs.com/Uncle-Guang/p/8796507.html)<br>
　　　　　[timeit 模块详解（准确测量小段代码的执行时间）](https://blog.csdn.net/qq_41556318/article/details/84671367)<br>
　　　　　[python性能分析(一) | timeit模块](https://blog.csdn.net/tylitianrui/article/details/83009188)<br>
　　　　　[Python timeit模块原理及使用方法](https://www.jb51.net/article/197127.htm)<br>

### 一、简介
　　timeit 模块提供了测量 Python 小段代码执行时间的方法。它既可以在命令行界面直接使用，也可以通过导入模块进行调用。该模块灵活地避开了测试执行时间所容易出现的错误。<br>
　　timeit 模块是 Python 标准库中的模块，无需安装，直接导入就可以使用。可以使用 timeit() 函数和 repeat() 函数，还有 Timer 类。使用 from timeit import ... 时，只能导入 Timer 类(有全局变量 __all__ 限制)。timeit 模块的源码总共只有 300 多行，主要就是实现上面的两个函数和一个类。<br>
### 二、使用
#### 2.1、命令行界面的使用方法：
　　当被作为命令行程序调用时，可以使用下列选项：
>python -m timeit [-n N] [-r N] [-s S] [-t] [-c] [-h] [statement ...]

　　各个选项的含义：

|选项|原型|含义|
|:---|:---|:---|
|-n N|--number=N|	执行指定语句（段）的次数|
|-r N|--repeat=N|	重复测量的次数（默认 3 次）|
|-s S|--setup=S|	指定初始化代码或构建环境的导入语句（默认是 pass）|
|-p|--process|	测量进程时间而不是实际执行时间（使用 time.process_time() 代替默认的 time.perf_counter()）|	 	 
|-t|--time|	使用 time.time()（不推荐）|
|-c|--clock|	使用 time.clock()（不推荐）|
|-v|--verbose|	打印原始的计时结果，输出更大精度的数值|
|-h|--help|	打印一个简短的用法信息并退出|

>\\$ python -m timeit '"-".join(str(n) for n in range(100))'<br>
10000 loops, best of 3: 40.3 usec per loop<br>
\\$ python -m timeit '"-".join(\[str(n) for n in range(100)\])'<br>
10000 loops, best of 3: 33.4 usec per loop<br>
\\$ python -m timeit '"-".join(map(str, range(100)))'<br>
10000 loops, best of 3: 25.2 usec per loop<br>

以下演示如果在开始的时候设置初始化语句：
>\\$ python -m timeit -s 'text = "I love FishC.com!"; char = "o"'  'char in text'
10000000 loops, best of 3: 0.0877 usec per loop
\\$ python -m timeit -s 'text = "I love FishC.com!"; char = "o"'  'text.find(char)'
1000000 loops, best of 3: 0.342 usec per loop

#### 2.2、timeit 模块
　　IDLE 下调用的方法：<br>
>import timeit<br>
timeit.timeit('"-".join(str(n) for n in range(100))', number=10000)<br>
0.8187260627746582<br>
timeit.timeit('"-".join(\[str(n) for n in range(100)\])', number=10000)<br>
0.7288308143615723<br>
timeit.timeit('"-".join(map(str, range(100)))', number=10000)<br>
0.5858950614929199<br>

需要注意的是，只有当使用命令行界面时，timeit 才会自动确定重复的次数(默认1000000次)。<br>

#### 2.3、该模块定义了三个实用函数和一个公共类。
#### 2.3.1
>timeit.timeit(stmt='pass', setup='pass', timer=<default timer>, number=1000000)<br>

　　功能：计算语句执行 number 次的时间。<br>
　　创建一个 Timer 实例，参数分别是 stmt（需要测量的语句或函数），setup（初始化代码或构建环境的导入语句），timer（计时函数），number（每一次测量中语句被执行的次数），它会先执行一次 setup 参数的语句，然后计算 stmt 参数的语句执行 number 次的时间，返回值是以秒为单位的浮点数。number 参数的默认值是一百万，stmt、setup 和 timer 参数由 timeit.Timer 类的构造函数传递。<br>
　　注：由于 timeit() 正在执行语句，语句中如果存在返回值的话会阻止 timeit() 返回执行时间。timeit() 会取代原语句中的返回值。<br>
> import timeit<br>
timeit.timeit('char in text', setup='text = "I love FishC.com!"; char = "o"')<br>
0.41440500499993504<br>
timeit.timeit('text.find(char)', setup='text = "I love FishC.com!"; char = "o"')<br>

　　注：默认情况下，timeit() 在计时的时候会暂时关闭 Python 的垃圾回收机制。这样做的优点是计时结果更具有可比性，但缺点是 GC（garbage collection，垃圾回收机制的缩写）有时候是测量函数性能的一个重要组成部分。如果是这样的话，GC 可以在 setup 参数执行第一条语句的时候被重新启动，例如：
>timeit.Timer('for i in range(10): oct(i)', 'gc.enable()').timeit()

#### 2.3.2
>timeit.repeat(stmt='pass', setup='pass', timer=<default timer>, repeat=3, number=1000000)<br>

　　功能：重复调用 timeit()。<br>
　　创建一个 Timer 实例，参数分别是 stmt（需要测试的函数或语句，字符串形式），setup（setup：传入 stmt 的运行环境，如 stmt 中使用到的参数、变量，要导入的模块等），timer（计时函数），repeat（重复测量的次数），number（每一次测量中语句被执行的次数）。repeat() 方法相当于持续多次调用 timeit() 方法并将结果返回为一个列表。repeat 参数指定重复的次数，number 参数传递给 timeit() 方法的 number 参数。<br>
　　注意：人们很容易计算出平均值和标准偏差，但这并不是非常有用。在典型的情况下，最低值取决于你的机器可以多快地运行给定的代码段；在结果中更高的那些值通常不是由于 Python 的速度导致，而是因为其他进程干扰了你的计时精度。所以，你所应感兴趣的只有结果的最低值（可以用 min() 求出）。<br>
  
#### 2.3.3
>timeit.default_timer()<br>

　　默认的计时器，一般是 time.perf_counter()，time.perf_counter() 方法能够在任一平台提供最高精度的计时器（它也只是记录了自然时间，记录自然时间会被很多其他因素影响，例如计算机的负载）。<br>
  
#### 2.3.4
>class timeit.Timer(stmt='pass', setup='pass', timer=<timer function>)<br> 

　　计算小段代码执行速度的类，构造函数需要的参数有 stmt（需要测量的语句或函数），setup（初始化代码或构建环境的导入语句），timer（计时函数）。前两个参数的默认值都是 'pass'，timer 参数是平台相关的；前两个参数都可以包含多个语句，多个语句间使用分号（;）或新行分隔开。<br>
　　第一次测试语句的时间，可以使用 timeit() 方法；repeat() 方法相当于持续多次调用 timeit() 方法并将结果返回为一个列表。<br>
　　stmt 和 setup 参数也可以是可供调用但没有参数的对象，这将会在一个计时函数中嵌套调用它们，然后被 timeit() 所执行。<br>
> import timeit<br>
t = timeit.Timer('char in text', setup='text = "I love FishC.com!"; char = "o"')<br>
t.timeit()<br>
0.3955516149999312<br>
t.repeat()<br>
[0.40193588800002544, 0.3960157959998014, 0.39594301399984033]<br>

　　注意，由于额外的调用，计时开销会相对略掉。<br>

#### 2.3.5
>print_exc(file=None) 

　　功能：输出计时代码的回溯（Traceback）<br>
>t = Timer(...)       # outside the try/except<br>
try:<br>
　　t.timeit(...)    # or t.repeat(...)<br>
except Exception:<br>
　　t.print_exc()<br>

　　标准回溯的优点是在编译模板中，源语句行会被显示出来。可选的 file 参数指定将回溯发送的位置，默认是发送到 sys.stderr。

#### 2.3.6
　　为了使 timeit 模块可以测量你的函数，你可以在 setup 参数中通过 import 语句导入：<br>
>def test():<br>
　　"""Stupid test function"""<br>
　　L = [i for i in range(100)]<br>
 <br>
if __name__ == '__main__':<br>
　　import timeit<br>
　　print(timeit.timeit("test()", setup="from __main__ import test"))<br>
    
附上 timeti 模块的实现源代码，小甲鱼强烈建议有时间的朋友可以研究一下（这对你的编程能力大有裨益）：

